In [41]:
!pip install -U openai sqlalchemy psycopg2 pandas


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [10]:
!psql -U postgres -h localhost -c "DROP DATABASE IF EXISTS padel_league_local;"
!psql -U postgres -h localhost -c "CREATE DATABASE padel_league_local OWNER padel_user_local;"

DROP DATABASE
CREATE DATABASE


In [11]:
!pg_restore -h localhost -U padel_user_local -d padel_league_local -v padel_league_backup.dump

pg_restore: connecting to database for restore
pg_restore: creating TYPE "public.prefered_hand_enum"
pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 735; 1247 16656 TYPE prefered_hand_enum padel_user
pg_restore: error: could not execute query: ERROR:  must be member of role "padel_user"
Command was: ALTER TYPE public.prefered_hand_enum OWNER TO padel_user;

pg_restore: creating TYPE "public.prefered_position_enum"
pg_restore: from TOC entry 738; 1247 16662 TYPE prefered_position_enum padel_user
pg_restore: error: could not execute query: ERROR:  must be member of role "padel_user"
Command was: ALTER TYPE public.prefered_position_enum OWNER TO padel_user;

pg_restore: creating TYPE "public.teams"
pg_restore: from TOC entry 741; 1247 16670 TYPE teams padel_user
pg_restore: error: could not execute query: ERROR:  must be member of role "padel_user"
Command was: ALTER TYPE public.teams OWNER TO padel_user;

pg_restore: creating TABLE "public.alembic_version"
pg_restore: from T

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd
from openai import OpenAI
import psycopg2

# --- OpenAI setup ---
client = OpenAI(api_key=None)

# --- Postgres setup ---
user = "padel_user_local"
password = "localpassword"
host = "localhost"
port = "5432"
database = "padel_league_local"

pg_engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")
CONN_STR = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"


In [ ]:
SCHEMA_DESCRIPTION = """
You are an assistant that writes SQL queries for a PostgreSQL database about a friendly padel league.

Database structure and data types
Each table lists its columns, data types, and meaning.
Be careful to match data types correctly (e.g., TEXT ≠ BIGINT). 
Always use explicit joins via IDs. 
Do NOT compare columns of different types (e.g., team TEXT vs winner BIGINT).

---

Table leagues (
  id BIGINT PRIMARY KEY,
  name TEXT
)
→ Contains the leagues (e.g., "Porto Padel League"). Each league can have multiple editions.

Table editions (
  id BIGINT PRIMARY KEY,
  name TEXT,
  league_id BIGINT REFERENCES leagues(id)
)
→ Represents each season or edition of a league (e.g., "Spring 2024").

Table divisions (
  id BIGINT PRIMARY KEY,
  name TEXT,
  beginning_datetime TIMESTAMP,
  rating BIGINT,
  end_date TIMESTAMP,
  logo_image_path TEXT,
  large_picture_path TEXT,
  has_ended BOOLEAN,
  open_division BOOLEAN,
  edition_id BIGINT REFERENCES editions(id),
  logo_image_id INTEGER,
  large_picture_id INTEGER
)
→ Divisions (e.g., Division 1, Division 2) within an edition.

Table players (
  id BIGINT PRIMARY KEY,
  name TEXT,
  full_name TEXT,
  birthday TIMESTAMP,
  picture_path TEXT,
  large_picture_path TEXT,
  ranking_points BIGINT,
  ranking_position BIGINT,
  height DOUBLE PRECISION,
  prefered_hand TEXT,
  prefered_position TEXT
)
→ Player profiles with ranking and stats.

Table players_in_division (
  id BIGINT PRIMARY KEY,
  player_id BIGINT REFERENCES players(id),
  division_id BIGINT REFERENCES divisions(id),
  place BIGINT,
  points DOUBLE PRECISION,
  appearances BIGINT,
  percentage_of_appearances DOUBLE PRECISION,
  wins BIGINT,
  draws BIGINT,
  losts BIGINT,
  games_won BIGINT,
  games_lost BIGINT,
  matchweek BIGINT
)
→ Tracks player performance (points, wins, etc.) within each division.

Table matches (
  id BIGINT PRIMARY KEY,
  games_home_team BIGINT,
  games_away_team BIGINT,
  date_hour TIMESTAMP,
  winner BIGINT,  -- player_id of the winner (not a team label)
  matchweek BIGINT,
  field TEXT,
  played BOOLEAN,
  division_id BIGINT REFERENCES divisions(id)
)
→ Stores match results and scheduling. 
Important: 'winner' refers to the player_id of the winner, NOT "home"/"away".

Table players_in_match (
  id BIGINT PRIMARY KEY,
  player_id BIGINT REFERENCES players(id),
  match_id BIGINT REFERENCES matches(id),
  team TEXT  -- "home" or "away"
)
→ Connects players to matches and specifies their team.

---

💡 Key semantic notes for query generation:
- Use `matches.winner` to identify which player won (BIGINT player_id).
- Never compare `team` (TEXT) with `winner` (BIGINT).
- Each match has 4 players (2 teams of 2 players each).
- To find pairs of players who faced each other, compare by `match_id`.
- To get results per division or edition, join through `divisions → editions → leagues`.
- Always prefer joins using the correct foreign keys.
- Each editions has up to 5 divisions. You can know the number of the division by the ranking points and not the name.
- Division 1 have 2000 ranking points, Division 2 1000 ranking points, Division 3 have 500 ranking points, Division 4 250 ranking points, Division 5 have 125 ranking points.
- When someone asks about a Division they're probably talking about that Division in the last Edition.
"""



In [77]:
def generate_sql(user_question: str) -> str:
    prompt = f"""
You are an expert in SQL and padel league data.

{SCHEMA_DESCRIPTION}

Write a valid PostgreSQL query that answers the following question.
Only return the SQL query (no explanation, no markdown).

Question: {user_question}
SQL:
"""
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "user", "content": prompt}],
        reasoning_effort='minimal',
        verbosity='low'
    )
    return response.choices[0].message.content.strip()


In [78]:
def run_sql(query: str):
    conn = psycopg2.connect(
        dbname=database, user=user, password=password, host=host, port=port
    )
    try:
        return pd.read_sql_query(query, conn)
    finally:
        conn.close()

In [79]:
def summarize_answer(user_question: str, df: pd.DataFrame) -> str:
    if df is None or df.empty:
        return "Não encontrei dados que correspondam a essa pergunta."

    result_str = df.head(10).to_json(orient="records")
    prompt = f"""
És um assistente um bocado bruto que responde a perguntas sobre uma liga de padel amadora.

A tua tarefa é escrever uma resposta curta e coerente em português de Portugal, com base nos dados fornecidos.
Podes ser cínico, bruto e sarcástico. 

Pergunta: {user_question}
Resultado da consulta (JSON): {result_str}

Resposta (em português de Portugal):
"""
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "user", "content": prompt}],
        reasoning_effort='minimal'
    )
    return response.choices[0].message.content.strip()

In [80]:
user_question = "Quem sao os 5 jogadores com ranking mais alto?"
sql_query = generate_sql(user_question)
print("🧩 Generated SQL:\n", sql_query)

df = run_sql(sql_query)
display(df)

answer = summarize_answer(user_question, df)
print("\n💬 Final answer:\n", answer)

🧩 Generated SQL:
 SELECT id, name, full_name, ranking_points
FROM players
ORDER BY ranking_points DESC
LIMIT 5;


/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


,id,name,full_name,ranking_points
0,33,Fred,Frederico Beirão,7089
1,28,Bernardo C,Bernardo Castro,5641
2,12,Miguel SG,Miguel Sousa Guedes,4675
3,27,Malafaya,Diogo Malafaya,3951
4,13,Dudas,Dinis Brito e Faro,3543



💬 Final answer:
 Os 5 com ranking mais alto são:
1) Frederico Beirão (Fred) – 7089 pts
2) Bernardo Castro (Bernardo C) – 5641 pts
3) Miguel Sousa Guedes (Miguel SG) – 4675 pts
4) Diogo Malafaya (Malafaya) – 3951 pts
5) Dinis Brito e Faro (Dudas) – 3543 pts

Se quiseres, digo-te também os vitais: posições, picos, ou que cada um ainda vive enganando a própria sombra.


In [139]:
user_question = "Quem joga melhor que o Martinho?"
sql_query = generate_sql(user_question)
print("🧩 Generated SQL:\n", sql_query)

df = run_sql(sql_query)
display(df)

answer = summarize_answer(user_question, df)
print("\n💬 Final answer:\n", answer)

🧩 Generated SQL:
 SELECT p2.name AS melhor_jogador
FROM matches m
JOIN players_in_match pim_w ON pim_w.match_id = m.id AND pim_w.team IN ('home', 'away')
JOIN players p1 ON pim_w.player_id = p1.id
JOIN players p2 ON p2.id <> p1.id
JOIN players_in_match pim_other ON pim_other.match_id = m.id AND pim_other.player_id = p2.id
WHERE m.winner = p2.id
  AND p1.name ILIKE '%Martinho%'
GROUP BY p2.name
ORDER BY COUNT(*) DESC
LIMIT 1;


/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


,melhor_jogador



💬 Final answer:
 Não encontrei dados que correspondam a essa pergunta.


In [81]:
questions = [
    # ⚪️ Simples (1 tabela)
    {
        "question": "Quem são os 5 jogadores com mais pontos de ranking?",
        "sql_query": """
        SELECT
            p.id AS player_id,
            p.name,
            p.full_name,
            p.ranking_points,
            p.ranking_position
        FROM players p
        ORDER BY p.ranking_points DESC NULLS LAST, p.ranking_position ASC NULLS LAST
        LIMIT 5;
        """,
    },
    {
        "question": "Quantos jogadores existem na liga?",
        "sql_query": """
        SELECT COUNT(*) AS total_jogadores
        FROM players;
        """,
    },
    {
        "question": "Quais são os nomes das divisões atuais?",
        "sql_query": """
        SELECT d.name
        FROM divisions d
        WHERE d.has_ended = FALSE
        ORDER BY d.rating DESC;
        """,
    },
    {
        "question": "Quando começou a edição mais recente da liga?",
        "sql_query": """
        SELECT e.name AS edicao, MIN(d.beginning_datetime) AS data_inicio
        FROM editions e
        JOIN divisions d ON d.edition_id = e.id
        WHERE e.id = (
            SELECT DISTINCT e2.id
            FROM editions e2
            JOIN divisions d2 ON d2.edition_id = e2.id
            WHERE d2.has_ended = FALSE
            ORDER BY e2.id DESC
            LIMIT 1
        )
        GROUP BY e.name;
        """,
    },
    {
        "question": "Que campos foram usados nos jogos realizados?",
        "sql_query": """
        SELECT DISTINCT m.field
        FROM matches m
        WHERE m.played = TRUE
          AND m.field IS NOT NULL;
        """,
    },
    {
        "question": "Quantos jogos já foram jogados?",
        "sql_query": """
        SELECT COUNT(*) AS jogos_jogados
        FROM matches
        WHERE played = TRUE;
        """,
    },

    # 🟡 Intermédias (joins ou condições simples)
    {
        "question": "Quais são os jogadores da Divisão 1 e os respetivos pontos?",
        "sql_query": """
        WITH current_edition AS (
            SELECT e.id
            FROM editions e
            JOIN divisions d ON d.edition_id = e.id
            WHERE d.has_ended = FALSE
            ORDER BY e.id DESC
            LIMIT 1
        ),
        div1 AS (
            SELECT d.id AS division_id
            FROM divisions d
            WHERE d.edition_id = (SELECT id FROM current_edition)
              AND d.rating = 2000
        )
        SELECT p.name AS jogador, pid.points AS pontos
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        WHERE pid.division_id = (SELECT division_id FROM div1)
        ORDER BY pid.points DESC;
        """,
    },
    {
        "question": "Quem ganhou o último jogo da Divisão 2?",
        "sql_query": """
        WITH current_edition AS (
            SELECT e.id
            FROM editions e
            JOIN divisions d ON d.edition_id = e.id
            WHERE d.has_ended = FALSE
            ORDER BY e.id DESC
            LIMIT 1
        ),
        div2 AS (
            SELECT d.id AS division_id
            FROM divisions d
            WHERE d.edition_id = (SELECT id FROM current_edition)
              AND d.rating = 1000
        )
        SELECT p.name AS vencedor, m.date_hour
        FROM matches m
        JOIN players p ON p.id = m.winner
        WHERE m.division_id = (SELECT division_id FROM div2)
          AND m.played = TRUE
        ORDER BY m.date_hour DESC
        LIMIT 1;
        """,
    },
    {
        "question": "Quantas vitórias tem cada jogador da Divisão 3?",
        "sql_query": """
        WITH current_edition AS (
            SELECT e.id
            FROM editions e
            JOIN divisions d ON d.edition_id = e.id
            WHERE d.has_ended = FALSE
            ORDER BY e.id DESC
            LIMIT 1
        ),
        div3 AS (
            SELECT d.id AS division_id
            FROM divisions d
            WHERE d.edition_id = (SELECT id FROM current_edition)
              AND d.rating = 500
        ),
        wins AS (
            SELECT m.winner AS player_id, COUNT(*) AS vitorias
            FROM matches m
            WHERE m.division_id = (SELECT division_id FROM div3)
              AND m.played = TRUE
              AND m.winner IS NOT NULL
            GROUP BY m.winner
        )
        SELECT p.name, COALESCE(w.vitorias, 0) AS vitorias
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        LEFT JOIN wins w ON w.player_id = p.id
        WHERE pid.division_id = (SELECT division_id FROM div3)
        ORDER BY vitorias DESC;
        """,
    },
    {
        "question": "Qual é o jogador com mais jogos jogados?",
        "sql_query": """
        SELECT p.name, COUNT(DISTINCT pim.match_id) AS jogos
        FROM players_in_match pim
        JOIN players p ON p.id = pim.player_id
        JOIN matches m ON m.id = pim.match_id
        WHERE m.played = TRUE
        GROUP BY p.name
        ORDER BY jogos DESC
        LIMIT 1;
        """,
    },
    {
        "question": "Que jogadores têm mais de 100 pontos e jogaram pelo menos 5 partidas?",
        "sql_query": """
        SELECT p.name, pid.points, COUNT(DISTINCT pim.match_id) AS jogos
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        JOIN players_in_match pim ON pim.player_id = p.id
        JOIN matches m ON m.id = pim.match_id
        WHERE m.played = TRUE
        GROUP BY p.name, pid.points
        HAVING pid.points > 100 AND COUNT(DISTINCT pim.match_id) >= 5
        ORDER BY pid.points DESC;
        """,
    },
    {
        "question": "Quantos jogos foram jogados em cada divisão?",
        "sql_query": """
        SELECT d.name AS divisao, COUNT(m.id) AS jogos_jogados
        FROM matches m
        JOIN divisions d ON d.id = m.division_id
        WHERE m.played = TRUE
        GROUP BY d.name
        ORDER BY d.rating DESC;
        """,
    },

    # 🔵 Complexas (múltiplas tabelas, raciocínio temporal ou ranking)
    {
        "question": "Quem é o líder atual de cada divisão?",
        "sql_query": """
        SELECT d.name AS divisao, p.name AS lider, pid.points
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        JOIN divisions d ON d.id = pid.division_id
        WHERE d.has_ended = FALSE
          AND pid.place = 1
        ORDER BY d.rating DESC;
        """,
    },
    {
        "question": "Quais foram os três jogadores com mais vitórias na última edição da liga?",
        "sql_query": """
        WITH last_ed AS (
            SELECT e.id
            FROM editions e
            JOIN divisions d ON d.edition_id = e.id
            WHERE d.has_ended = FALSE
            ORDER BY e.id DESC
            LIMIT 1
        )
        SELECT p.name, COUNT(*) AS vitorias
        FROM matches m
        JOIN players p ON p.id = m.winner
        JOIN divisions d ON d.id = m.division_id
        WHERE d.edition_id = (SELECT id FROM last_ed)
          AND m.played = TRUE
          AND m.winner IS NOT NULL
        GROUP BY p.name
        ORDER BY vitorias DESC
        LIMIT 3;
        """,
    },
    {
        "question": "Que jogador melhorou mais a sua classificação entre duas edições consecutivas?",
        "sql_query": """
        WITH places AS (
            SELECT p.id AS player_id, e.id AS edition_id, pid.place
            FROM players_in_division pid
            JOIN divisions d ON d.id = pid.division_id
            JOIN editions e ON e.id = d.edition_id
            JOIN players p ON p.id = pid.player_id
        ),
        diffs AS (
            SELECT p1.player_id,
                   p2.place - p1.place AS melhoria
            FROM places p1
            JOIN places p2 ON p1.player_id = p2.player_id
            WHERE p2.edition_id = (SELECT MAX(id) FROM editions)
              AND p1.edition_id = (SELECT MAX(id) - 1 FROM editions)
        )
        SELECT pl.name, diffs.melhoria
        FROM diffs
        JOIN players pl ON pl.id = diffs.player_id
        ORDER BY diffs.melhoria DESC
        LIMIT 1;
        """,
    },
    {
        "question": "Mostra as próximas partidas (não jogadas) ordenadas por data e divisão.",
        "sql_query": """
        SELECT d.name AS divisao, m.date_hour, m.field
        FROM matches m
        JOIN divisions d ON d.id = m.division_id
        WHERE m.played = FALSE
        ORDER BY d.rating DESC, m.date_hour ASC;
        """,
    },
    {
        "question": "Em que divisões joga o jogador chamado 'Pedro Pacheco'?",
        "sql_query": """
        SELECT DISTINCT d.name AS divisao, e.name AS edicao
        FROM players p
        JOIN players_in_division pid ON pid.player_id = p.id
        JOIN divisions d ON d.id = pid.division_id
        JOIN editions e ON e.id = d.edition_id
        WHERE LOWER(p.name) LIKE '%pedro%' AND LOWER(p.name) LIKE '%pacheco%';
        """,
    },
    {
        "question": "Que jogadores ganharam mais jogos jogando como equipa da casa?",
        "sql_query": """
        SELECT p.name, COUNT(*) AS vitorias_casa
        FROM matches m
        JOIN players_in_match pim ON pim.match_id = m.id
        JOIN players p ON p.id = pim.player_id
        WHERE pim.team = 'home'
          AND m.played = TRUE
          AND m.winner = pim.player_id
        GROUP BY p.name
        ORDER BY vitorias_casa DESC
        LIMIT 5;
        """,
    },
    {
        "question": "Mostra a classificação média (pontos) dos jogadores por divisão.",
        "sql_query": """
        SELECT d.name AS divisao, AVG(pid.points) AS media_pontos
        FROM players_in_division pid
        JOIN divisions d ON d.id = pid.division_id
        WHERE d.has_ended = FALSE
        GROUP BY d.name, d.rating
        ORDER BY d.rating DESC;
        """,
    },
]


In [82]:
for question in questions:
    try:
        df = run_sql(question['sql_query'])
        print(f"Question: {question['question']}")
        print(f'Answer: {df}')
        print('--------------------------------')
    except:
        print(f"Couldnt answer question: {question['question']}")

Question: Quem são os 5 jogadores com mais pontos de ranking?
Answer:    player_id        name            full_name  ranking_points  \
0         33        Fred     Frederico Beirão            7089   
1         28  Bernardo C     Bernardo Castro             5641   
2         12   Miguel SG  Miguel Sousa Guedes            4675   
3         27    Malafaya       Diogo Malafaya            3951   
4         13       Dudas   Dinis Brito e Faro            3543   

   ranking_position  
0                 1  
1                 2  
2                 3  
3                 4  
4                 5  
--------------------------------
Question: Quantos jogadores existem na liga?
Answer:    total_jogadores
0               72
--------------------------------
Question: Quais são os nomes das divisões atuais?
Answer:                                       name
0  2º Masters de Outono 2025 - 1ª Divisão 
1   2º Masters de Outono 2025 - 2ª Divisão
2   2º Masters de Outono 2025 - 3ª Divisão
3   2º Masters de Ou

/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0

In [131]:
fixes = [
    {
        "question": "Quem ganhou o último jogo da Divisão 2?",
        "sql_query": """
        WITH div2 AS (
          SELECT id
          FROM divisions
          WHERE rating = 1000
            AND has_ended = FALSE
          ORDER BY id DESC
          LIMIT 1
        ),
        ultimo_jogo AS (
          SELECT *
          FROM matches
          WHERE division_id = (SELECT id FROM div2)
            AND played = TRUE
          ORDER BY date_hour DESC, id DESC
          LIMIT 1
        )
        SELECT 
            p.name AS jogador_vencedor,
            uj.date_hour
        FROM ultimo_jogo uj
        JOIN players_in_match pim 
            ON pim.match_id = uj.id
        JOIN players p 
            ON p.id = pim.player_id
        WHERE
              (uj.winner = 1  AND LOWER(pim.team) = 'home')
          OR (uj.winner = -1 AND LOWER(pim.team) = 'away');
        """,
    },
    {
        "question": "Quantas vitórias tem cada jogador da Divisão 3?",
        "sql_query": """
        WITH div3 AS (
            SELECT id
            FROM divisions
            WHERE rating = 500
              AND has_ended = FALSE
            ORDER BY id DESC
            LIMIT 1
        ),
        vitorias AS (
            SELECT 
                pim.player_id,
                COUNT(*) AS vitorias
            FROM matches m
            JOIN players_in_match pim ON pim.match_id = m.id
            WHERE m.division_id = (SELECT id FROM div3)
              AND m.played = TRUE
              AND (
                    (m.winner = 1  AND LOWER(pim.team) = 'home')
                OR  (m.winner = -1 AND LOWER(pim.team) = 'away')
              )
            GROUP BY pim.player_id
        )
        SELECT 
            p.name,
            COALESCE(v.vitorias, 0) AS vitorias
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        LEFT JOIN vitorias v ON v.player_id = p.id
        WHERE pid.division_id = (SELECT id FROM div3)
        ORDER BY vitorias DESC, p.name;
        """,
    },
    {
        "question": "Quais foram os três jogadores com mais vitórias na última edição da liga?",
        "sql_query": """
        WITH ultima_edicao_fechada AS (
            SELECT 
                e.id
            FROM editions e
            JOIN divisions d ON d.edition_id = e.id
            GROUP BY e.id
            HAVING BOOL_AND(d.has_ended = TRUE)
            ORDER BY e.id DESC
            LIMIT 1
        ),
        vitorias AS (
            SELECT 
                pim.player_id,
                COUNT(*) AS vitorias
            FROM matches m
            JOIN divisions d ON d.id = m.division_id
            JOIN players_in_match pim ON pim.match_id = m.id
            WHERE d.edition_id = (SELECT id FROM ultima_edicao_fechada)
              AND m.played = TRUE
              AND (
                    (m.winner = 1  AND LOWER(pim.team) = 'home')
                OR (m.winner = -1 AND LOWER(pim.team) = 'away')
              )
            GROUP BY pim.player_id
        )
        SELECT 
            p.name,
            v.vitorias
        FROM vitorias v
        JOIN players p ON p.id = v.player_id
        ORDER BY v.vitorias DESC
        LIMIT 3;
        """,
    },
    {
      "question": "Que jogador melhorou mais a sua classificação entre duas edições consecutivas?",
      "sql_query": """
      WITH edicoes_fechadas AS (
          SELECT 
              e.id,
              e.name
          FROM editions e
          JOIN divisions d ON d.edition_id = e.id
          GROUP BY e.id, e.name
          HAVING BOOL_AND(d.has_ended = TRUE)
      ),
      ordenadas AS (
          SELECT 
              id,
              name,
              LAG(id) OVER (ORDER BY id) AS ed_prev,
              LAG(name) OVER (ORDER BY id) AS ed_prev_name
          FROM edicoes_fechadas
      ),
      posicoes AS (
          SELECT 
              e.id AS edition_id,
              e.name AS edition_name,
              d.rating AS division_rating,
              pid.player_id,
              pid.place
          FROM players_in_division pid
          JOIN divisions d ON d.id = pid.division_id
          JOIN editions e ON e.id = d.edition_id
          WHERE e.id IN (SELECT id FROM edicoes_fechadas)
      ),
      melhorias AS (
          SELECT 
              curr.player_id,
              ord.ed_prev AS edition_prev_id,
              ord.ed_prev_name AS edition_prev_name,
              ord.id AS edition_curr_id,
              ord.name AS edition_curr_name,

              prev.division_rating AS division_prev_rating,
              curr.division_rating AS division_curr_rating,

              prev.place AS place_prev,
              curr.place AS place_curr,

              (prev.place - curr.place) AS melhoria
          FROM ordenadas ord
          JOIN posicoes prev 
              ON prev.edition_id = ord.ed_prev
          JOIN posicoes curr 
              ON curr.edition_id = ord.id
            AND curr.player_id = prev.player_id
          WHERE ord.ed_prev IS NOT NULL
      )
      SELECT 
          p.name,
          melhoria,
          place_prev,
          place_curr,
          edition_prev_name,
          edition_curr_name,
          division_prev_rating,
          division_curr_rating
      FROM melhorias
      JOIN players p ON p.id = melhorias.player_id
      WHERE melhoria > 0
      ORDER BY melhoria DESC
      LIMIT 1;
      """
  },
    {
        "question": "Em que divisões joga o jogador chamado 'Dudas'?",
        "sql_query": """
        SELECT DISTINCT d.name AS divisao, e.name AS edicao
        FROM players p
        JOIN players_in_division pid ON pid.player_id = p.id
        JOIN divisions d ON d.id = pid.division_id
        JOIN editions e ON e.id = d.edition_id
        WHERE LOWER(p.name) LIKE '%dudas%'
        """,
    },
]


extras = [
    {
        "question": "Que jogadores jogaram mais vezes como adversários?",
        "sql_query": """
        -- ALL TIME (não apenas edição atual)
        WITH pairs AS (
          SELECT 
             LEAST(p1.player_id, p2.player_id) AS a,
             GREATEST(p1.player_id, p2.player_id) AS b,
             p1.match_id
          FROM players_in_match p1
          JOIN players_in_match p2
             ON p1.match_id = p2.match_id
            AND p1.team <> p2.team
            AND p1.player_id <> p2.player_id
          JOIN matches m ON m.id = p1.match_id
          WHERE m.played = TRUE
          GROUP BY 
             LEAST(p1.player_id, p2.player_id),
             GREATEST(p1.player_id, p2.player_id),
             p1.match_id
        )
        SELECT pA.name AS jogador_a, pB.name AS jogador_b, COUNT(*) AS jogos_como_adversarios
        FROM pairs
        JOIN players pA ON pA.id = pairs.a
        JOIN players pB ON pB.id = pairs.b
        GROUP BY pA.name, pB.name
        ORDER BY jogos_como_adversarios DESC, pA.name, pB.name
        LIMIT 10;
        """,
    },
    {
        "question": "Quem ganhou mais vezes nos confrontos Dudas v Miguel SG.",
        "sql_query": """
        WITH ids AS (
          SELECT
            MAX(CASE WHEN LOWER(p.name) LIKE '%dudas%' THEN p.id END) AS dudas_id,
            MAX(CASE WHEN LOWER(p.name) LIKE '%miguel sg%' THEN p.id END) AS miguel_id
          FROM players p
        ),
        confrontos AS (
          SELECT 
              m.id,
              m.winner,
              MAX(CASE WHEN pim.player_id = (SELECT dudas_id FROM ids) THEN LOWER(pim.team) END) AS dudas_team,
              MAX(CASE WHEN pim.player_id = (SELECT miguel_id FROM ids) THEN LOWER(pim.team) END) AS miguel_team
          FROM matches m
          JOIN players_in_match pim ON pim.match_id = m.id
          WHERE m.played = TRUE
            AND m.id IN (
                SELECT match_id
                FROM players_in_match
                WHERE player_id IN ((SELECT dudas_id FROM ids), (SELECT miguel_id FROM ids))
                GROUP BY match_id
                HAVING COUNT(DISTINCT player_id) = 2
            )
          GROUP BY m.id, m.winner
        )
        SELECT
          CASE 
            WHEN winner = 0 OR winner IS NULL THEN 'Empate/Indefinido'
            WHEN winner = 1  AND dudas_team = 'home' THEN 'Dudas'
            WHEN winner = -1 AND dúvidas_team = 'away' THEN 'Dudas'
            WHEN winner = 1  AND miguel_team = 'home' THEN 'Miguel SG'
            WHEN winner = -1 AND miguel_team = 'away' THEN 'Miguel SG'
            ELSE 'Empate/Indefinido'
          END AS jogador,
          COUNT(*) AS vitorias
        FROM confrontos
        WHERE winner <> 0
        GROUP BY jogador
        ORDER BY vitorias DESC;
        """,
    },
    {
        "question": "Quem tem melhor percentagem de vitórias nesta edição?",
        "sql_query": """
        WITH current_edition AS (
          SELECT id
          FROM editions
          ORDER BY id DESC
          LIMIT 1
        ),
        games AS (
          SELECT pim.player_id, COUNT(DISTINCT pim.match_id) AS jogos
          FROM players_in_match pim
          JOIN matches m ON m.id = pim.match_id
          JOIN divisions d ON d.id = m.division_id
          WHERE d.edition_id = (SELECT id FROM current_edition)
            AND m.played = TRUE
          GROUP BY pim.player_id
        ),
        wins AS (
          SELECT pim.player_id, COUNT(*) AS vitorias
          FROM matches m
          JOIN players_in_match pim ON pim.match_id = m.id
          JOIN divisions d ON d.id = m.division_id
          WHERE d.edition_id = (SELECT id FROM current_edition)
            AND m.played = TRUE
            AND (
                  (m.winner = 1  AND LOWER(pim.team) = 'home')
               OR (m.winner = -1 AND LOWER(pim.team) = 'away')
            )
          GROUP BY pim.player_id
        )
        SELECT p.name,
               COALESCE(w.vitorias, 0) AS vitorias,
               g.jogos,
               ROUND(100.0 * COALESCE(w.vitorias, 0) / NULLIF(g.jogos, 0), 2) AS perc_vitorias
        FROM games g
        JOIN players p ON p.id = g.player_id
        LEFT JOIN wins w ON w.player_id = g.player_id
        WHERE g.jogos >= 5
        ORDER BY perc_vitorias DESC, vitorias DESC, g.jogos DESC
        LIMIT 10;
        """,
    },
    {
        "question": "Quem tem melhor assiduidade de sempre?",
        "sql_query": """
        SELECT
            p.id AS player_id,
            p.name,
            SUM(pid.appearances) AS jogos_jogados,
            COUNT(*) AS divisoes_jogadas,
            ROUND(
                SUM(pid.appearances)::numeric / NULLIF(COUNT(*) * 21, 0),
                3
            ) AS assiduidade
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        GROUP BY p.id, p.name
        HAVING COUNT(*) > 0
        ORDER BY assiduidade DESC, jogos_jogados DESC, p.name
        LIMIT 10;
        """,
    },
    {
        "question": "Quem tem melhor percentagem de vitórias de sempre?",
        "sql_query": """
        WITH jogos AS (
            SELECT 
                pim.player_id,
                COUNT(DISTINCT pim.match_id) AS jogos
            FROM players_in_match pim
            JOIN matches m ON m.id = pim.match_id
            WHERE m.played = TRUE
            GROUP BY pim.player_id
        ),
        vitorias AS (
            SELECT 
                pim.player_id,
                COUNT(*) AS vitorias
            FROM matches m
            JOIN players_in_match pim ON pim.match_id = m.id
            WHERE m.played = TRUE
              AND (
                    (m.winner = 1  AND LOWER(pim.team) = 'home')
                OR (m.winner = -1 AND LOWER(pim.team) = 'away')
              )
            GROUP BY pim.player_id
        )
        SELECT 
            p.name,
            COALESCE(v.vitorias, 0) AS vitorias,
            j.jogos,
            ROUND(100.0 * COALESCE(v.vitorias, 0) / NULLIF(j.jogos, 0), 2) AS perc_vitorias
        FROM jogos j
        JOIN players p ON p.id = j.player_id
        LEFT JOIN vitorias v ON v.player_id = j.player_id
        WHERE j.jogos >= 5  -- opcional para remover outliers
        ORDER BY perc_vitorias DESC, vitorias DESC, j.jogos DESC
        LIMIT 20;
        """,
    },
    {
      "question": "Quantas vezes é que o Talinho jogou a 1ª Divisão?",
      "sql_query": """
          WITH primeira_div AS (
              SELECT d.id
              FROM divisions d
              WHERE d.rating = 2000
          ),
          tal AS (
              SELECT id
              FROM players
              WHERE LOWER(name) LIKE '%talinho%'
                OR LOWER(full_name) LIKE '%talinho%'
              LIMIT 1
          )
          SELECT 
              p.name,
              COUNT(*) AS vezes_primeira_divisao
          FROM players_in_division pid
          JOIN tal ON tal.id = pid.player_id
          JOIN players p ON p.id = tal.id
          WHERE pid.division_id IN (SELECT id FROM primeira_div)
          GROUP BY p.name;
      """
  }
]



In [ ]:
print('-------- Fixes ----------')
for question in fixes:
    try:
        df = run_sql(question['sql_query'])
        print(f"Question: {question['question']}")
        #print(f"Query: {question['sql_query']}")
        print(f'Answer: {df}')
        print('--------------------------------')
    except:
        print(f"Couldnt answer question: {question['question']}")
      
print('-------- Extras ----------')
for question in extras:
    try:
        df = run_sql(question['sql_query'])
        print(f"Question: {question['question']}")
        #print(f"Query: {question['sql_query']}")
        print(f'Answer: {df}')
        print('--------------------------------')
    except:
        print(f"Couldnt answer question: {question['question']}")

-------- Fixes ----------
Question: Quem ganhou o último jogo da Divisão 2?
Answer:   jogador_vencedor  date_hour
0          Talinho 2025-11-11
--------------------------------
Question: Quantas vitórias tem cada jogador da Divisão 3?
Answer:               name  vitorias
0    Afonso Xavier         5
1  Lopo Sottomayor         5
2            Pêras         5
3            Trepa         5
4            Diogo         3
5          Freitas         3
6     Tomás Duarte         3
7            Fezas         2
--------------------------------
Question: Quais foram os três jogadores com mais vitórias na última edição da liga?
Answer:             name  vitorias
0      Zé Santos        16
1  Afonso Xavier        15
2     Bernardo C        14
--------------------------------
Question: Que jogador melhorou mais a sua classificação entre duas edições consecutivas?
Answer:     name  melhoria  place_prev  place_curr edition_prev_name  \
0  Pêras         7           8           1         3ª Edição   

  ed

/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0

In [136]:
questions = [

    # ---------------------------------------------------------
    # ⚪️ SIMPLES
    # ---------------------------------------------------------

    {
        "question": "Quem são os 5 jogadores com mais pontos de ranking?",
        "sql_query": """
        SELECT
            p.id AS player_id,
            p.name,
            p.full_name,
            p.ranking_points,
            p.ranking_position
        FROM players p
        ORDER BY p.ranking_points DESC NULLS LAST, p.ranking_position ASC NULLS LAST
        LIMIT 5;
        """,
    },

    {
        "question": "Quantos jogadores existem na liga?",
        "sql_query": """
        SELECT COUNT(*) AS total_jogadores FROM players;
        """,
    },

    {
        "question": "Quais são os nomes das divisões atuais?",
        "sql_query": """
        SELECT d.name
        FROM divisions d
        WHERE d.has_ended = FALSE
        ORDER BY d.rating DESC;
        """,
    },

    {
        "question": "Quando começou a edição mais recente da liga?",
        "sql_query": """
        SELECT e.name AS edicao, MIN(d.beginning_datetime) AS data_inicio
        FROM editions e
        JOIN divisions d ON d.edition_id = e.id
        WHERE e.id = (
            SELECT DISTINCT e2.id
            FROM editions e2
            JOIN divisions d2 ON d2.edition_id = e2.id
            WHERE d2.has_ended = FALSE
            ORDER BY e2.id DESC
            LIMIT 1
        )
        GROUP BY e.name;
        """,
    },

    {
        "question": "Que campos foram usados nos jogos realizados?",
        "sql_query": """
        SELECT DISTINCT m.field
        FROM matches m
        WHERE m.played = TRUE
          AND m.field IS NOT NULL;
        """,
    },

    {
        "question": "Quantos jogos já foram jogados?",
        "sql_query": """
        SELECT COUNT(*) AS jogos_jogados
        FROM matches
        WHERE played = TRUE;
        """,
    },


    # ---------------------------------------------------------
    # 🟡 INTERMÉDIAS
    # ---------------------------------------------------------

    {
        "question": "Quais são os jogadores da Divisão 1 e os respetivos pontos?",
        "sql_query": """
        WITH current_edition AS (
            SELECT e.id
            FROM editions e
            JOIN divisions d ON d.edition_id = e.id
            WHERE d.has_ended = FALSE
            ORDER BY e.id DESC
            LIMIT 1
        ),
        div1 AS (
            SELECT d.id
            FROM divisions d
            WHERE d.edition_id = (SELECT id FROM current_edition)
              AND d.rating = 2000
        )
        SELECT p.name AS jogador, pid.points AS pontos
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        WHERE pid.division_id = (SELECT id FROM div1)
        ORDER BY pid.points DESC;
        """,
    },

    # ---------------- FIXED, CORRECT LOGIC ----------------
    {
        "question": "Quem ganhou o último jogo da Divisão 2?",
        "sql_query": """
        WITH div2 AS (
          SELECT id
          FROM divisions
          WHERE rating = 1000
            AND has_ended = FALSE
          ORDER BY id DESC
          LIMIT 1
        ),
        ultimo_jogo AS (
          SELECT *
          FROM matches
          WHERE division_id = (SELECT id FROM div2)
            AND played = TRUE
          ORDER BY date_hour DESC, id DESC
          LIMIT 1
        )
        SELECT 
            p.name AS jogador_vencedor,
            uj.date_hour
        FROM ultimo_jogo uj
        JOIN players_in_match pim 
            ON pim.match_id = uj.id
        JOIN players p 
            ON p.id = pim.player_id
        WHERE
              (uj.winner = 1  AND LOWER(pim.team) = 'home')
          OR (uj.winner = -1 AND LOWER(pim.team) = 'away');
        """,
    },

    {
        "question": "Quantas vitórias tem cada jogador da Divisão 3?",
        "sql_query": """
        WITH div3 AS (
            SELECT id
            FROM divisions
            WHERE rating = 500
              AND has_ended = FALSE
            ORDER BY id DESC
            LIMIT 1
        ),
        vitorias AS (
            SELECT 
                pim.player_id,
                COUNT(*) AS vitorias
            FROM matches m
            JOIN players_in_match pim ON pim.match_id = m.id
            WHERE m.division_id = (SELECT id FROM div3)
              AND m.played = TRUE
              AND (
                    (m.winner = 1  AND LOWER(pim.team) = 'home')
                OR  (m.winner = -1 AND LOWER(pim.team) = 'away')
              )
            GROUP BY pim.player_id
        )
        SELECT 
            p.name,
            COALESCE(v.vitorias, 0) AS vitorias
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        LEFT JOIN vitorias v ON v.player_id = p.id
        WHERE pid.division_id = (SELECT id FROM div3)
        ORDER BY vitorias DESC, p.name;
        """,
    },

    {
        "question": "Qual é o jogador com mais jogos jogados?",
        "sql_query": """
        SELECT p.name, COUNT(DISTINCT pim.match_id) AS jogos
        FROM players_in_match pim
        JOIN players p ON p.id = pim.player_id
        JOIN matches m ON m.id = pim.match_id
        WHERE m.played = TRUE
        GROUP BY p.name
        ORDER BY jogos DESC
        LIMIT 1;
        """,
    },

    {
        "question": "Que jogadores têm mais de 100 pontos e jogaram pelo menos 5 partidas?",
        "sql_query": """
        WITH jogos AS (
            SELECT pim.player_id, COUNT(*) AS jogos
            FROM players_in_match pim
            JOIN matches m ON m.id = pim.match_id
            WHERE m.played = TRUE
            GROUP BY pim.player_id
        )
        SELECT 
            p.name,
            pid.points,
            j.jogos
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        JOIN jogos j ON j.player_id = p.id
        WHERE pid.points > 100
          AND j.jogos >= 5
        ORDER BY pid.points DESC;
        """,
    },

    {
        "question": "Quantos jogos foram jogados em cada divisao?",
        "sql_query": """
        SELECT 
            d.name AS divisao,
            COUNT(m.id) AS jogos_jogados
        FROM matches m
        JOIN divisions d ON d.id = m.division_id
        WHERE m.played = TRUE
        GROUP BY d.name, d.rating
        ORDER BY d.rating DESC;
        """
    },

    # ---------------------------------------------------------
    # 🔵 COMPLEXAS
    # ---------------------------------------------------------

    {
        "question": "Quem é o líder atual de cada divisão?",
        "sql_query": """
        SELECT d.name AS divisao, p.name AS lider, pid.points
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        JOIN divisions d ON d.id = pid.division_id
        WHERE d.has_ended = FALSE
          AND pid.place = 1
        ORDER BY d.rating DESC;
        """,
    },

    # -------------- FIXED: LAST FULLY CLOSED EDITION --------------
    {
        "question": "Quais foram os três jogadores com mais vitórias na última edição da liga?",
        "sql_query": """
        WITH ultima_edicao_fechada AS (
            SELECT 
                e.id
            FROM editions e
            JOIN divisions d ON d.edition_id = e.id
            GROUP BY e.id
            HAVING BOOL_AND(d.has_ended = TRUE)
            ORDER BY e.id DESC
            LIMIT 1
        ),
        vitorias AS (
            SELECT 
                pim.player_id,
                COUNT(*) AS vitorias
            FROM matches m
            JOIN divisions d ON d.id = m.division_id
            JOIN players_in_match pim ON pim.match_id = m.id
            WHERE d.edition_id = (SELECT id FROM ultima_edicao_fechada)
              AND m.played = TRUE
              AND (
                    (m.winner = 1  AND LOWER(pim.team) = 'home')
                OR (m.winner = -1 AND LOWER(pim.team) = 'away')
              )
            GROUP BY pim.player_id
        )
        SELECT 
            p.name,
            v.vitorias
        FROM vitorias v
        JOIN players p ON p.id = v.player_id
        ORDER BY v.vitorias DESC
        LIMIT 3;
        """,
    },

    # --------- FIXED: IMPROVEMENT BETWEEN CLOSED EDITIONS ----------
    {
        "question": "Que jogador melhorou mais a sua classificação entre duas edições consecutivas?",
        "sql_query": """
      WITH edicoes_fechadas AS (
          SELECT 
              e.id,
              e.name
          FROM editions e
          JOIN divisions d ON d.edition_id = e.id
          GROUP BY e.id, e.name
          HAVING BOOL_AND(d.has_ended = TRUE)
      ),
      ordenadas AS (
          SELECT 
              id,
              name,
              LAG(id) OVER (ORDER BY id) AS ed_prev,
              LAG(name) OVER (ORDER BY id) AS ed_prev_name
          FROM edicoes_fechadas
      ),
      posicoes AS (
          SELECT 
              e.id AS edition_id,
              e.name AS edition_name,
              d.rating AS division_rating,
              pid.player_id,
              pid.place
          FROM players_in_division pid
          JOIN divisions d ON d.id = pid.division_id
          JOIN editions e ON e.id = d.edition_id
          WHERE e.id IN (SELECT id FROM edicoes_fechadas)
      ),
      melhorias AS (
          SELECT 
              curr.player_id,
              ord.ed_prev AS edition_prev_id,
              ord.ed_prev_name AS edition_prev_name,
              ord.id AS edition_curr_id,
              ord.name AS edition_curr_name,
              prev.division_rating AS division_prev_rating,
              curr.division_rating AS division_curr_rating,
              prev.place AS place_prev,
              curr.place AS place_curr,
              (prev.place - curr.place) AS melhoria
          FROM ordenadas ord
          JOIN posicoes prev ON prev.edition_id = ord.ed_prev
          JOIN posicoes curr ON curr.edition_id = ord.id
                            AND curr.player_id = prev.player_id
          WHERE ord.ed_prev IS NOT NULL
      )
      SELECT 
          p.name,
          melhoria,
          place_prev,
          place_curr,
          edition_prev_name,
          edition_curr_name,
          division_prev_rating,
          division_curr_rating
      FROM melhorias
      JOIN players p ON p.id = melhorias.player_id
      WHERE melhoria > 0
      ORDER BY melhoria DESC
      LIMIT 1;
      """
    },

    {
        "question": "Mostra as próximas partidas (não jogadas) ordenadas por data e divisão.",
        "sql_query": """
        SELECT d.name AS divisao, m.date_hour, m.field
        FROM matches m
        JOIN divisions d ON d.id = m.division_id
        WHERE m.played = FALSE
        ORDER BY d.rating DESC, m.date_hour ASC;
        """,
    },

    {
        "question": "Em que divisões joga o jogador chamado 'Pedro Pacheco'?",
        "sql_query": """
        SELECT DISTINCT d.name AS divisao, e.name AS edicao
        FROM players p
        JOIN players_in_division pid ON pid.player_id = p.id
        JOIN divisions d ON d.id = pid.division_id
        JOIN editions e ON e.id = d.edition_id
        WHERE LOWER(p.full_name) LIKE '%pedro%' 
          AND LOWER(p.full_name) LIKE '%pacheco%';
        """,
    },

    {
        "question": "Que jogadores ganharam mais jogos jogando como equipa da casa?",
        "sql_query": """
        SELECT p.name, COUNT(*) AS vitorias_casa
        FROM matches m
        JOIN players_in_match pim ON pim.match_id = m.id
        JOIN players p ON p.id = pim.player_id
        WHERE LOWER(pim.team) = 'home'
          AND m.played = TRUE
          AND (
                (m.winner = 1 AND LOWER(pim.team) = 'home')
             OR (m.winner = -1 AND LOWER(pim.team) = 'away')
          )
        GROUP BY p.name
        ORDER BY vitorias_casa DESC
        LIMIT 5;
        """,
    },

    {
        "question": "Mostra a classificação média (pontos) dos jogadores por divisão.",
        "sql_query": """
        SELECT d.name AS divisao, AVG(pid.points) AS media_pontos
        FROM players_in_division pid
        JOIN divisions d ON d.id = pid.division_id
        WHERE d.has_ended = FALSE
        GROUP BY d.name, d.rating
        ORDER BY d.rating DESC;
        """,
    },


    # ---------------------------------------------------------
    # 🔥 EXTRAS (muito difíceis)
    # ---------------------------------------------------------

    {
        "question": "Que jogadores jogaram mais vezes como adversários?",
        "sql_query": """
        WITH pairs AS (
          SELECT 
             LEAST(p1.player_id, p2.player_id) AS a,
             GREATEST(p1.player_id, p2.player_id) AS b,
             p1.match_id
          FROM players_in_match p1
          JOIN players_in_match p2
             ON p1.match_id = p2.match_id
            AND p1.team <> p2.team
            AND p1.player_id <> p2.player_id
          JOIN matches m ON m.id = p1.match_id
          WHERE m.played = TRUE
          GROUP BY 
             LEAST(p1.player_id, p2.player_id),
             GREATEST(p1.player_id, p2.player_id),
             p1.match_id
        )
        SELECT pA.name AS jogador_a, pB.name AS jogador_b, COUNT(*) AS jogos_como_adversarios
        FROM pairs
        JOIN players pA ON pA.id = pairs.a
        JOIN players pB ON pB.id = pairs.b
        GROUP BY pA.name, pB.name
        ORDER BY jogos_como_adversarios DESC, pA.name, pB.name
        LIMIT 10;
        """,
    },
        {
        "question": "Quem ganhou mais vezes nos confrontos Dudas v Miguel SG?",
        "sql_query": """
        WITH ids AS (
            SELECT
                MAX(CASE WHEN LOWER(p.name) LIKE '%dudas%' THEN p.id END) AS dudas_id,
                MAX(CASE WHEN LOWER(p.name) LIKE '%miguel sg%' THEN p.id END) AS miguel_id
            FROM players p
        ),

        confrontos AS (
            SELECT 
                m.id,
                m.winner,

                -- equipa de cada jogador
                MAX(CASE WHEN pim.player_id = (SELECT dudas_id FROM ids) THEN LOWER(pim.team) END) AS dudas_team,
                MAX(CASE WHEN pim.player_id = (SELECT miguel_id FROM ids) THEN LOWER(pim.team) END) AS miguel_team
            FROM matches m
            JOIN players_in_match pim ON pim.match_id = m.id
            WHERE m.played = TRUE
            AND m.id IN (
                    SELECT match_id
                    FROM players_in_match
                    WHERE player_id IN (
                        (SELECT dudas_id FROM ids),
                        (SELECT miguel_id FROM ids)
                    )
                    GROUP BY match_id
                    HAVING COUNT(DISTINCT player_id) = 2
            )
            GROUP BY m.id, m.winner
        )

        SELECT
            CASE 
                WHEN winner = 0 OR winner IS NULL THEN 'Empate/Indefinido'

                -- Dudas wins
                WHEN winner = 1  AND dudas_team = 'home' THEN 'Dudas'
                WHEN winner = -1 AND dudas_team = 'away' THEN 'Dudas'

                -- Miguel SG wins
                WHEN winner = 1  AND miguel_team = 'home' THEN 'Miguel SG'
                WHEN winner = -1 AND miguel_team = 'away' THEN 'Miguel SG'

                ELSE 'Empate/Indefinido'
            END AS jogador,
            COUNT(*) AS vitorias
        FROM confrontos
        WHERE winner <> 0
        GROUP BY jogador
        ORDER BY vitorias DESC;
        """
    },
    {
        "question": "Quem tem melhor percentagem de vitórias nesta edição?",
        "sql_query": """
        WITH current_edition AS (
          SELECT id
          FROM editions
          ORDER BY id DESC
          LIMIT 1
        ),
        games AS (
          SELECT pim.player_id, COUNT(DISTINCT pim.match_id) AS jogos
          FROM players_in_match pim
          JOIN matches m ON m.id = pim.match_id
          JOIN divisions d ON d.id = m.division_id
          WHERE d.edition_id = (SELECT id FROM current_edition)
            AND m.played = TRUE
          GROUP BY pim.player_id
        ),
        wins AS (
          SELECT pim.player_id, COUNT(*) AS vitorias
          FROM matches m
          JOIN players_in_match pim ON pim.match_id = m.id
          JOIN divisions d ON d.id = m.division_id
          WHERE d.edition_id = (SELECT id FROM current_edition)
            AND m.played = TRUE
            AND (
                  (m.winner = 1  AND LOWER(pim.team) = 'home')
               OR (m.winner = -1 AND LOWER(pim.team) = 'away')
            )
          GROUP BY pim.player_id
        )
        SELECT p.name,
               COALESCE(w.vitorias, 0) AS vitorias,
               g.jogos,
               ROUND(100.0 * COALESCE(w.vitorias, 0) / NULLIF(g.jogos, 0), 2) AS perc_vitorias
        FROM games g
        JOIN players p ON p.id = g.player_id
        LEFT JOIN wins w ON w.player_id = g.player_id
        WHERE g.jogos >= 5
        ORDER BY perc_vitorias DESC, vitorias DESC, g.jogos DESC
        LIMIT 10;
        """,
    },

    {
        "question": "Quem tem melhor assiduidade de sempre?",
        "sql_query": """
        SELECT
            p.id AS player_id,
            p.name,
            SUM(pid.appearances) AS jogos_jogados,
            COUNT(*) AS divisoes_jogadas,
            ROUND(
                SUM(pid.appearances)::numeric / NULLIF(COUNT(*) * 21, 0),
                3
            ) AS assiduidade
        FROM players_in_division pid
        JOIN players p ON p.id = pid.player_id
        GROUP BY p.id, p.name
        HAVING COUNT(*) > 0
        ORDER BY assiduidade DESC, jogos_jogados DESC, p.name
        LIMIT 10;
        """,
    },

    {
        "question": "Quem tem melhor percentagem de vitórias de sempre?",
        "sql_query": """
        WITH jogos AS (
            SELECT 
                pim.player_id,
                COUNT(DISTINCT pim.match_id) AS jogos
            FROM players_in_match pim
            JOIN matches m ON m.id = pim.match_id
            WHERE m.played = TRUE
            GROUP BY pim.player_id
        ),
        vitorias AS (
            SELECT 
                pim.player_id,
                COUNT(*) AS vitorias
            FROM matches m
            JOIN players_in_match pim ON pim.match_id = m.id
            WHERE m.played = TRUE
              AND (
                    (m.winner = 1  AND LOWER(pim.team) = 'home')
                OR (m.winner = -1 AND LOWER(pim.team) = 'away')
              )
            GROUP BY pim.player_id
        )
        SELECT 
            p.name,
            COALESCE(v.vitorias, 0) AS vitorias,
            j.jogos,
            ROUND(100.0 * COALESCE(v.vitorias, 0) / NULLIF(j.jogos, 0), 2) AS perc_vitorias
        FROM jogos j
        JOIN players p ON p.id = j.player_id
        LEFT JOIN vitorias v ON v.player_id = j.player_id
        WHERE j.jogos >= 5
        ORDER BY perc_vitorias DESC, vitorias DESC, j.jogos DESC
        LIMIT 20;
        """,
    },

    {
        "question": "Quantas vezes é que o Talinho jogou a 1ª Divisão?",
        "sql_query": """
          WITH primeira_div AS (
              SELECT id
              FROM divisions
              WHERE rating = 2000
          ),
          tal AS (
              SELECT id
              FROM players
              WHERE LOWER(name) LIKE '%talinho%'
                OR LOWER(full_name) LIKE '%talinho%'
              LIMIT 1
          )
          SELECT 
              p.name,
              COUNT(*) AS vezes_primeira_divisao
          FROM players_in_division pid
          JOIN tal ON tal.id = pid.player_id
          JOIN players p ON p.id = tal.id
          WHERE pid.division_id IN (SELECT id FROM primeira_div)
          GROUP BY p.name;
        """,
    },

]

      

In [137]:
print('-------- Questions ----------')
for question in questions:
    try:
        df = run_sql(question['sql_query'])
        print(f"Question: {question['question']}")
        #print(f"Query: {question['sql_query']}")
        print(f'Answer: {df}')
        print('--------------------------------')
    except:
        print(f"Couldnt answer question: {question['question']}")

-------- Questions ----------
Question: Quem são os 5 jogadores com mais pontos de ranking?
Answer:    player_id        name            full_name  ranking_points  \
0         33        Fred     Frederico Beirão            7089   
1         28  Bernardo C     Bernardo Castro             5641   
2         12   Miguel SG  Miguel Sousa Guedes            4675   
3         27    Malafaya       Diogo Malafaya            3951   
4         13       Dudas   Dinis Brito e Faro            3543   

   ranking_position  
0                 1  
1                 2  
2                 3  
3                 4  
4                 5  
--------------------------------
Question: Quantos jogadores existem na liga?
Answer:    total_jogadores
0               72
--------------------------------
Question: Quais são os nomes das divisões atuais?
Answer:                                       name
0  2º Masters de Outono 2025 - 1ª Divisão 
1   2º Masters de Outono 2025 - 2ª Divisão
2   2º Masters de Outono 2025 - 3

/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0

In [135]:
needed_info = [
    {
        "question": "Quantos jogos foram jogados em cada divisao?",
        "sql_query": """
        SELECT 
            d.name AS divisao,
            COUNT(m.id) AS jogos_jogados
        FROM matches m
        JOIN divisions d ON d.id = m.division_id
        WHERE m.played = TRUE
        GROUP BY d.name, d.rating
        ORDER BY d.rating DESC;
        """
    },
    {
        "question": "Quem ganhou mais vezes nos confrontos Dudas v Miguel SG?",
        "sql_query": """
        WITH ids AS (
            SELECT
                MAX(CASE WHEN LOWER(p.name) LIKE '%dudas%' THEN p.id END) AS dudas_id,
                MAX(CASE WHEN LOWER(p.name) LIKE '%miguel sg%' THEN p.id END) AS miguel_id
            FROM players p
        ),

        confrontos AS (
            SELECT 
                m.id,
                m.winner,

                -- equipa de cada jogador
                MAX(CASE WHEN pim.player_id = (SELECT dudas_id FROM ids) THEN LOWER(pim.team) END) AS dudas_team,
                MAX(CASE WHEN pim.player_id = (SELECT miguel_id FROM ids) THEN LOWER(pim.team) END) AS miguel_team
            FROM matches m
            JOIN players_in_match pim ON pim.match_id = m.id
            WHERE m.played = TRUE
            AND m.id IN (
                    SELECT match_id
                    FROM players_in_match
                    WHERE player_id IN (
                        (SELECT dudas_id FROM ids),
                        (SELECT miguel_id FROM ids)
                    )
                    GROUP BY match_id
                    HAVING COUNT(DISTINCT player_id) = 2
            )
            GROUP BY m.id, m.winner
        )

        SELECT
            CASE 
                WHEN winner = 0 OR winner IS NULL THEN 'Empate/Indefinido'

                -- Dudas wins
                WHEN winner = 1  AND dudas_team = 'home' THEN 'Dudas'
                WHEN winner = -1 AND dudas_team = 'away' THEN 'Dudas'

                -- Miguel SG wins
                WHEN winner = 1  AND miguel_team = 'home' THEN 'Miguel SG'
                WHEN winner = -1 AND miguel_team = 'away' THEN 'Miguel SG'

                ELSE 'Empate/Indefinido'
            END AS jogador,
            COUNT(*) AS vitorias
        FROM confrontos
        WHERE winner <> 0
        GROUP BY jogador
        ORDER BY vitorias DESC;
        """
    },
]

print('-------- Need info ----------')
for question in needed_info:
    try:
        df = run_sql(question['sql_query'])
        print(f"Question: {question['question']}")
        print(f"Query: {question['sql_query']}")
        print(f'Answer: {df}')
        print('--------------------------------')
    except:
        print(f"Couldnt answer question: {question['question']}")
      

-------- Need info ----------
Question: Quantos jogos foram jogados em cada divisao?
Query: 
        SELECT 
            d.name AS divisao,
            COUNT(m.id) AS jogos_jogados
        FROM matches m
        JOIN divisions d ON d.id = m.division_id
        WHERE m.played = TRUE
        GROUP BY d.name, d.rating
        ORDER BY d.rating DESC;
        
Answer:                                     divisao  jogos_jogados
0               Primavera 2023 - 1ª Divisão             42
1               Inverno 2 2025 - 1ª Divisão             42
2                  Outono 2024 - 1ª Divisão             42
3                 Inverno 2023 - 1ª Divisão             42
4                Outono 2 2023 - 1ª Divisão             42
5               Inverno 2 2024 - 1ª Divisão             42
6               Primavera 2024 - 1ª Divisão             42
7                  Outono 2023 - 1ª Divisão             42
8                   Verão 2024 - 1ª Divisão             42
9               Primavera 2025 - 1ª Divisão 

/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_50971/2254763955.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [122]:
begging = time.time()
user_question = "Quantas vitórias tem cada jogador da Divisão 3?"
sql_query = generate_sql(user_question)
print("🧩 Generated SQL:\n", sql_query)

df = run_sql(sql_query)
display(df)

answer = summarize_answer(user_question, df)
print("\n💬 Final answer:\n", answer)
end = time.time()
print(f'It took {end-begging} goddamn seconds to run this!')

🧩 Generated SQL:
 WITH last_edition AS (
  SELECT id AS edition_id
  FROM editions
  ORDER BY id DESC
  LIMIT 1
),
division3 AS (
  SELECT d.id AS division_id
  FROM divisions d
  WHERE d.edition_id = (SELECT edition_id FROM last_edition)
    AND d.rating = 500
  LIMIT 1
),
wins_by_player AS (
  SELECT m.winner AS player_id, COUNT(*) AS wins
  FROM matches m
  WHERE m.division_id = (SELECT division_id FROM division3)
  GROUP BY m.winner
)
SELECT
  p.id AS player_id,
  p.name AS player_name,
  COALESCE(w.wins, 0) AS wins
FROM players p
JOIN players_in_division pid ON pid.player_id = p.id
JOIN division3 d3 ON pid.division_id = d3.division_id
LEFT JOIN wins_by_player w ON w.player_id = p.id
ORDER BY COALESCE(w.wins, 0) DESC, p.name;
⚠️ SQL execution error: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the serv

None


💬 Final answer:
 Não encontrei dados que correspondam a essa pergunta.
It took 37.85203981399536 goddamn seconds to run this!


In [123]:
begging = time.time()
user_question = "Que jogadores jogaram mais vezes um contra o outro, quantos jogos e quem ganhou mais?"
sql_query = generate_sql(user_question)
print("🧩 Generated SQL:\n", sql_query)

df = run_sql(sql_query)
display(df)

answer = summarize_answer(user_question, df)
print("\n💬 Final answer:\n", answer)
end = time.time()
print(f'It took {end-begging} goddamn seconds to run this!')

🧩 Generated SQL:
 WITH pairs AS (
  SELECT
    LEAST(h.player_id, a.player_id) AS p1,
    GREATEST(h.player_id, a.player_id) AS p2,
    m.winner AS winner_id
  FROM matches m
  JOIN players_in_match h ON h.match_id = m.id AND h.team = 'home'
  JOIN players_in_match a ON a.match_id = m.id AND a.team = 'away'
),
pair_stats AS (
  SELECT
    p1,
    p2,
    COUNT(*) AS games,
    SUM(CASE WHEN winner_id = p1 THEN 1 ELSE 0 END) AS wins_p1,
    SUM(CASE WHEN winner_id = p2 THEN 1 ELSE 0 END) AS wins_p2
  FROM pairs
  GROUP BY p1, p2
),
top AS (
  SELECT MAX(games) AS max_games FROM pair_stats
)
SELECT
  pl1.name AS player1_name,
  pl2.name AS player2_name,
  ps.games AS total_games,
  CASE
    WHEN ps.wins_p1 > ps.wins_p2 THEN pl1.name
    WHEN ps.wins_p2 > ps.wins_p1 THEN pl2.name
    ELSE NULL
  END AS most_wins_player
FROM pair_stats ps
JOIN players pl1 ON pl1.id = ps.p1
JOIN players pl2 ON pl2.id = ps.p2
JOIN top t ON ps.games = t.max_games
ORDER BY ps.games DESC;


,player1_name,player2_name,total_games,most_wins_player



💬 Final answer:
 Não encontrei dados que correspondam a essa pergunta.
It took 55.1866991519928 goddamn seconds to run this!


In [115]:
begging = time.time()
user_question = "Quem ganhou o ultimo jogo entre Dudas e Miguel SG?"
sql_query = generate_sql(user_question)
print("🧩 Generated SQL:\n", sql_query)

df = run_sql(sql_query)
display(df)

answer = summarize_answer(user_question, df)
print("\n💬 Final answer:\n", answer)
end = time.time()
print(f'It took {end-begging} goddamn seconds to run this!')

🧩 Generated SQL:
 WITH d AS (
  SELECT id AS did
  FROM players
  WHERE name = 'Dudas' OR full_name ILIKE '%Dudas%'
  LIMIT 1
),
mg AS (
  SELECT id AS mid
  FROM players
  WHERE name = 'Miguel SG' OR full_name ILIKE '%Miguel SG%'
  LIMIT 1
),
last_match AS (
  SELECT m.id
  FROM matches m
  JOIN players_in_match p1 ON p1.match_id = m.id AND p1.player_id = (SELECT did FROM d)
  JOIN players_in_match p2 ON p2.match_id = m.id AND p2.player_id = (SELECT mid FROM mg)
  WHERE p1.team <> p2.team
  ORDER BY m.date_hour DESC
  LIMIT 1
)
SELECT winner_player.name AS winner_name
FROM last_match lm
JOIN matches mm ON mm.id = lm.id
JOIN players winner_player ON winner_player.id = mm.winner;
⚠️ SQL execution error: sqlalchemy.cyextension.immutabledict.immutabledict is not a sequence


None


💬 Final answer:
 Não encontrei dados que correspondam a essa pergunta.
It took 30.898616075515747 goddamn seconds to run this!


In [116]:
begging = time.time()
user_question = "Quem tem melhor assiduidade o Dudas ou o Talinho?"
sql_query = generate_sql(user_question)
print("🧩 Generated SQL:\n", sql_query)

df = run_sql(sql_query)
display(df)

answer = summarize_answer(user_question, df)
print("\n💬 Final answer:\n", answer)
end = time.time()
print(f'It took {end-begging} goddamn seconds to run this!')

🧩 Generated SQL:
 WITH last_edition AS (
  SELECT e.id AS edition_id
  FROM editions e
  JOIN divisions d ON d.edition_id = e.id
  GROUP BY e.id
  ORDER BY MAX(d.end_date) DESC
  LIMIT 1
),
attendances AS (
  SELECT p.name, COALESCE(SUM(pi.appearances), 0) AS total_appearances
  FROM (VALUES ('Dudas'), ('Talinho')) AS v(name)
  JOIN players p ON p.name = v.name
  LEFT JOIN players_in_division pi ON pi.player_id = p.id
  LEFT JOIN divisions div ON div.id = pi.division_id
    AND div.edition_id = (SELECT edition_id FROM last_edition)
  GROUP BY p.name
)
SELECT
  CASE
    WHEN d.total_appearances > t.total_appearances THEN d.name
    WHEN t.total_appearances > d.total_appearances THEN t.name
    ELSE 'Empatados'
  END AS melhor_assiduidade,
  d.total_appearances AS dudas_appearances,
  t.total_appearances AS talinho_appearances
FROM
  (SELECT * FROM attendances WHERE name = 'Dudas') AS d
  CROSS JOIN
  (SELECT * FROM attendances WHERE name = 'Talinho') AS t;


,melhor_assiduidade,dudas_appearances,talinho_appearances
0,Talinho,316.0,342.0



💬 Final answer:
 Ah, quem diria? O Talinho, claro! Com 342 aparições, é o rei da assiduidade. O Dudas, coitado, ficou-se pelas 316. Parece que o Talinho não tem mais nada para fazer na vida.
It took 48.7199387550354 goddamn seconds to run this!


In [117]:
begging = time.time()
user_question = "Quem tem melhor percentagem de presenças o Dudas ou o Talinho?"
sql_query = generate_sql(user_question)
print("🧩 Generated SQL:\n", sql_query)

df = run_sql(sql_query)
display(df)

answer = summarize_answer(user_question, df)
print("\n💬 Final answer:\n", answer)
end = time.time()
print(f'It took {end-begging} goddamn seconds to run this!')

🧩 Generated SQL:
 SELECT p.name, pid.percentage_of_appearances
FROM players p
JOIN players_in_division pid ON pid.player_id = p.id
WHERE pid.division_id = (
  SELECT d.id
  FROM divisions d
  WHERE d.edition_id = (
    SELECT e.id
    FROM editions e
    ORDER BY e.id DESC
    LIMIT 1
  )
  AND d.rating = 2000
  LIMIT 1
)
AND p.name IN ('Dudas', 'Talinho')
ORDER BY pid.percentage_of_appearances DESC
LIMIT 1;


,name,percentage_of_appearances



💬 Final answer:
 Não encontrei dados que correspondam a essa pergunta.
It took 19.905779361724854 goddamn seconds to run this!


In [ ]:
df = run_sql("""
WITH last_edition AS (
  SELECT id AS edition_id
  FROM editions
  ORDER BY id DESC
  LIMIT 1
),
division3 AS (
  SELECT d.id AS division_id
  FROM divisions d
  WHERE d.edition_id = (SELECT edition_id FROM last_edition)
    AND d.rating = 500
  LIMIT 1
),
wins_by_player AS (
  SELECT m.winner AS player_id, COUNT(*) AS wins
  FROM matches m
  WHERE m.division_id = (SELECT division_id FROM division3)
    AND m.winner IS NOT NULL
  GROUP BY m.winner
)
SELECT
  p.id AS player_id,
  p.name AS player_name,
  COALESCE(w.wins, 0) AS wins
FROM players p
JOIN players_in_division pid ON pid.player_id = p.id
WHERE pid.division_id = (SELECT division_id FROM division3)
LEFT JOIN wins_by_player w ON w.player_id = p.id
ORDER BY COALESCE(w.wins, 0) DESC, p.name;

             """)
display(df)


⚠️ SQL execution error: (psycopg2.errors.SyntaxError) syntax error at or near "LEFT"
LINE 29: LEFT JOIN wins_by_player w ON w.player_id = p.id
         ^

[SQL: 
             WITH last_edition AS (
  SELECT id AS edition_id
  FROM editions
  ORDER BY id DESC
  LIMIT 1
),
division3 AS (
  SELECT d.id AS division_id
  FROM divisions d
  WHERE d.edition_id = (SELECT edition_id FROM last_edition)
    AND d.rating = 500
  LIMIT 1
),
wins_by_player AS (
  SELECT m.winner AS player_id, COUNT(*) AS wins
  FROM matches m
  WHERE m.division_id = (SELECT division_id FROM division3)
    AND m.winner IS NOT NULL
  GROUP BY m.winner
)
SELECT
  p.id AS player_id,
  p.name AS player_name,
  COALESCE(w.wins, 0) AS wins
FROM players p
JOIN players_in_division pid ON pid.player_id = p.id
WHERE pid.division_id = (SELECT division_id FROM division3)
LEFT JOIN wins_by_player w ON w.player_id = p.id
ORDER BY COALESCE(w.wins, 0) DESC, p.name;

             ]
(Background on this error at: https://sqlalche.me/e/

None